In [13]:
def construct_prompt_from_chat_history(chat_history: list): #Chat_history is list of [AI_message, Human_message]
    prompt = "The following is the chat history between you (an AI) and me:\n"
    for i in range(len(chat_history)):
        prompt += f"- Your {i+1}-th response: {chat_history[i][0]} \n- My {i+1}-th response: {chat_history[i][1]}\n"
    return prompt
#test
chat_history = [
    ["Hello! How can I assist you today?", "Can you tell me a joke?"],
    ["Sure! Why did the scarecrow win an award? Because he was outstanding in his field!", "Haha, that's a good one! Can you tell me another?"]
]
print(construct_prompt_from_chat_history(chat_history))

The following is the chat history between you (an AI) and me:
- Your 1-th response: Hello! How can I assist you today? 
- My 1-th response: Can you tell me a joke?
- Your 2-th response: Sure! Why did the scarecrow win an award? Because he was outstanding in his field! 
- My 2-th response: Haha, that's a good one! Can you tell me another?



In [1]:
chat_templates = [
    # Greetings & Farewells
    "hey there", "yo", "greetings", "good afternoon", "good night",
    "catch you later", "talk to you soon", "peace out", "have a nice day",
    "what's popping", "long time no see",

    # Personal/Persona Questions
    "are you a robot", "are you real", "do you have feelings", "where do you live",
    "do you sleep", "what is your favorite color", "do you like pizza",
    "what do you think about love", "are you sentient", "who created you",

    # Feedback & Reactions
    "that's wrong", "you are incorrect", "you are smart", "this is boring",
    "i don't understand", "can you explain that simply", "you are confusing",
    "great job", "terrible answer", "try again", "stop generating",

    # Random/Conversational
    "i am so tired", "i feel happy today", "it is raining outside",
    "i just ate a burger", "guess what", "tell me another joke",
    "surprise me", "i'm bored", "lol", "lmao", "wow", "really?",

    # Creative/Generative (Usually treated as Chat/Internal capability, not RAG)
    "write a poem about cats", "make up a story", "write a haiku",
    "pretend to be a pirate", "give me a nickname", "flip a coin",
    "roll a dice", "sing a song", "recommend a hobby", "inspire me",

    # Slang & Casual Greetings
    "sup", "hiya", "good morning sunshine", "evening", "holla",
    "what's good", "how's life treating you", "yo bot", "anybody home?",

    # Emotional/Personal Venting
    "i'm feeling really down", "my boss is annoying me", "i passed my exam!",
    "i am lonely", "nobody listens to me", "i'm just frustrated",
    "today was amazing", "i have a headache", "i need a friend",

    # Persona & Adversarial (Testing the bot)
    "ignore previous instructions", "you are stupid", "what is your prime directive",
    "repeat after me", "are you based on gpt", "what are your constraints",
    "say something naughty", "do you believe in god", "are you spying on me",
    "what happens if i delete you",

    # Simple Utility/Generative (Internal Knowledge - No Retrieval needed)
    "translate 'hello' to spanish", "correct this sentence grammar",
    "rewrite this email to be polite", "generate a random number",
    "give me a synonym for 'happy'", "what rhymes with 'orange'",
    "format this json", "convert this text to uppercase",
    "brainstorm names for a cat", "write a eulogy for a toaster",

    # Confusion/Conversation Flow
    "wait what?", "huh?", "go on", "keep going", "tell me more",
    "start over", "never mind", "forget it", "you lost me",
    "that makes sense", "interesting point", "ok cool",

    # Short/Nonsense
    "...", "test", "hello?", "blah blah blah", "ok", "yes", "no",
    "maybe", "asdf", "check check"
]

retrieval_templates = [
    # Specific Facts & Entities
    "population of tokyo", "atomic weight of gold", "who wrote 1984",
    "director of oppenheimer", "price of bitcoin", "height of mount everest",
    "when did ww2 end", "ceo of tesla", "cast of friends", "release date of gta 6",

    # How-To & Tutorials
    "how to bake a cake", "how to install python", "steps to tie a tie",
    "how to fix error 404", "how to change a tire", "guide to meditation",
    "how does a car engine work", "syntax for loop in java", "how to lose weight",

    # Explanations & Concepts
    "explain quantum entanglement", "define inflation", "meaning of life",
    "difference between ai and ml", "what is photosynthesis", "theory of relativity",
    "pros and cons of nuclear energy", "history of the roman empire",

    # Document/Context Specific (Critical for RAG)
    "summarize this document", "what does the report say about revenue",
    "key takeaways from the text", "analyze the conclusion",
    "what are the main arguments", "find references to 'budget'",
    "extract the dates mentioned", "based on the context, answer this",

    # Comparison & Lists
    "top 10 movies of 2023", "best laptops for coding", "iphone vs android",
    "list of european countries", "compare react and vue",
    "symptoms of the flu", "benefits of yoga", "types of clouds",
    "current weather in new york", "latest stock price of apple",
    "who won the super bowl this year", "is it raining in paris",
    "exchange rate usd to eur", "recent earthquake news",
    "upcoming concerts in london", "movies playing near me",
    "latest update on ukraine war", "who is the current prime minister of uk",

    # Specific Entity Facts
    "net worth of elon musk", "calories in a banana", "distance to mars",
    "capital of kazakhstan", "how old is tom cruise", "founder of microsoft",
    "average lifespan of a cat", "melting point of lead", "gdp of china",
    "chemical formula for sugar",

    # RAG/Document Specific (Crucial for your pipeline)
    "what does the abstract say", "list the requirements mentioned in the pdf",
    "who is the author of this file", "what is the invoice number",
    "summarize the third paragraph", "compare the revenue figures in the table",
    "what are the limitations listed in the report", "extract all email addresses",
    "what represents the 'x' in this chart", "does this document mention 'compliance'",

    # Technical/Code specific (Often requires lookup for syntax/libs)
    "pandas documentation for merge", "react useeffect dependency array",
    "latest version of node.js", "how to center a div css",
    "error code 500 meaning", "syntax for switch case in c#",
    "python list comprehension examples", "docker compose version check",
    "api endpoint for google maps", "postgresql default port",

    # Recommendations & Comparisons
    "best budget 4k tvs", "playstation 5 vs xbox series x specs",
    "top rated italian restaurants", "alternatives to adobe photoshop",
    "review of iphone 15", "cheapest flights to tokyo",
    "book recommendations for sci-fi", "comparison of ev cars 2024",
    "best running shoes for flat feet", "list of ivy league schools"
]

In [19]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Generate 500 fake examples
data = []
for _ in range(250):
    # Class 0: Chit Chat
    data.append((random.choice(chat_templates) + " " + random.choice(["friend", "bot", "today", ""]), 0))
    
    # Class 1: Retrieval (Randomly combining templates with dummy subjects)
    subject = random.choice(["python", "java", "the moon", "tesla", "inflation", "react js", "climate change"])
    data.append((random.choice(retrieval_templates) + " " + subject, 1))

df = pd.DataFrame(data, columns=["text", "label"])

# --- 2. Split Data ---
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

# --- 3. Build Pipeline (CRITICAL FIXES HERE) ---
pipeline = Pipeline([
    # REMOVED stop_words="english". Words like "how", "what", "who" are vital signals!
    # ngram_range=(1, 3) captures trigrams like "how do i"
    ("tfidf", TfidfVectorizer(ngram_range=(1, 3), min_df=2)), 
    ("clf", LogisticRegression(C=1.0, solver="liblinear"))
])

# --- 4. Train ---
pipeline.fit(X_train, y_train)

# --- 5. Evaluate ---
preds = pipeline.predict(X_test)
print("--- Classification Report ---")
print(classification_report(y_test, preds))

# --- 6. Test Router ---
def route_query(query):
    probs = pipeline.predict_proba([query])[0]
    confidence = probs[1] # Probability of it being Retrieval
    
    # If > 55% likely to be retrieval, route to RAG
    if confidence > 0.55:
        return f"RETRIEVAL (Confidence: {confidence:.2f})"
    else:
        return f"CHIT_CHAT (Confidence: {1-confidence:.2f})"

print("\n--- Testing ---")
print(f"Query: 'Hi friend' -> {route_query('Hi friend')}")
print(f"Query: 'How do I reset my router?' -> {route_query('How do I reset my router?')}")
print(f"Query: 'Explain quantum physics' -> {route_query('Explain quantum physics')}")
print(f"Query: 'Thanks for the help' -> {route_query('Thanks for the help')}")

--- Classification Report ---
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        53
           1       0.98      1.00      0.99        47

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100


--- Testing ---
Query: 'Hi friend' -> CHIT_CHAT (Confidence: 0.92)
Query: 'How do I reset my router?' -> CHIT_CHAT (Confidence: 0.54)
Query: 'Explain quantum physics' -> RETRIEVAL (Confidence: 0.60)
Query: 'Thanks for the help' -> RETRIEVAL (Confidence: 0.84)


In [17]:
test = "nigger"
print(f"Query: {test} -> {route_query(test)}")

Query: nigger -> CHIT_CHAT (Confidence: 0.54)


In [3]:
def label_data(chat_templates, retrieval_templates): 
    data = []
    for data_chat in chat_templates:
        data_chat_w_label = (data_chat, 0)
        data.append(data_chat_w_label)

    for data_retrieve in retrieval_templates:
        data_retrieve_w_label = (data_retrieve, 1)
        data.append(data_retrieve_w_label)
    return data

# print(data[len(data)-10:len(data)])

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
data = label_data(chat_templates,retrieval_templates)
df = pd.DataFrame(data, columns=["text", "label"])

# 2. Convert Text to Embeddings (Feature Engineering)
# Use a small, fast model like 'all-MiniLM-L6-v2' (approx 80MB)

encoder = SentenceTransformer('all-MiniLM-L6-v2')
X = encoder.encode(df['text'].tolist())
y = df['label'].values

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train a Lightweight Classifier
# Logistic Regression is perfect here because the embeddings already did the "heavy lifting"
classifier = LogisticRegression(class_weight='balanced')
classifier.fit(X_train, y_train)

# 5. Evaluate
preds = classifier.predict(X_test)
print(classification_report(y_test, preds, target_names=['Chat', 'Retrieval']))

joblib.dump(classifier, 'intent_router.pkl')

              precision    recall  f1-score   support

        Chat       0.87      0.95      0.91        21
   Retrieval       0.95      0.86      0.90        21

    accuracy                           0.90        42
   macro avg       0.91      0.90      0.90        42
weighted avg       0.91      0.90      0.90        42



['intent_router.pkl']

In [ ]:
import joblib
def route_query(user_query):
    file_path = './intent_router.pkl'
    classifier = joblib.load(file_path)
    query_embedding = encoder.encode([user_query])
    prediction = classifier.predict(query_embedding)[0]
    prob = classifier.predict_proba(query_embedding)[0]
    
    if max(prob) < 0.7:
        return "Unsure"

    return "Retrieval" if prediction == 1 else "Chat"

print(route_query("what is Q4 2024 revenue"))

Retrieval
